<a href="https://colab.research.google.com/github/sipocz/RevoText/blob/main/Zenga_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q playwright nest_asyncio
!playwright install chromium
import nest_asyncio
import asyncio
import pandas as pd
from playwright.async_api import async_playwright

nest_asyncio.apply()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 15.6 MB/s eta 0:00:00
172.5 MiB [] 0% 11.4s172.5 MiB [] 0% 19.6s172.5 MiB [] 0% 9.3s172.5 MiB [] 0% 6.5s172.5 MiB [] 1% 5.1s172.5 MiB [] 1% 5.6s172.5 MiB [] 1% 5.4s172.5 MiB [] 2% 5.9s172.5 MiB [] 2% 5.7s172.5 MiB [] 2% 5.2s172.5 MiB [] 3% 4.6s172.5 MiB [] 4% 4.3s172.5 MiB [] 5% 4.2s172.5 MiB [] 5% 4.6s172.5 MiB [] 6% 4.5s172.5 MiB [] 6% 4.6s172.5 MiB [] 7% 4.3s172.5 MiB [] 8% 4.2s172.5 MiB [] 8% 4.0s172.5 MiB [] 9% 4.0s172.5 MiB [] 10% 3.8s172.5 MiB [] 10% 3.7s172.5 MiB [] 10% 3.8s172.5 MiB [] 11% 3.7s172.5 MiB [] 11% 3.8s172.5 MiB [] 12% 3.6s172.5 MiB [] 13% 3.5s172.5 MiB [] 14% 3.4s172.5 MiB [] 14% 3.3s172.5 MiB [] 15% 3.3s172.5 MiB [] 16% 3.2s172.5 MiB [] 17% 3.1s172.5 MiB [] 17% 3.0s172.5 MiB [] 18% 3.0s172.5 MiB [] 19% 2.9s172.5 MiB [] 19% 2.8s172.5 MiB [] 20% 2.8s172.5 MiB [] 21% 2.7s172.5 MiB [] 22% 2.7s172.5 MiB [] 23% 2.6s172.5 MiB [] 24% 2.5s172.5 MiB [] 25% 2.5s172.5 MiB [] 26% 2.4s172.5 MiB [] 27% 2.3s172.5 MiB [] 28

In [2]:
_BASE_URL_='https://www.zenga.hu'
def url_creator(place:str="pest",type:str="haz"):

    return f"{_BASE_URL_}/{place}+elado+{type}?page="

In [3]:
url_creator()

'https://www.zenga.hu/pest+elado+haz?page='

In [4]:
async def scrape_zenga_links(start_url="https://www.zenga.hu/komarom-esztergom-megye+elado+haz?page=",page=1):

    start_url = start_url+str(page)
    links = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(start_url)
        await page.wait_for_timeout(8000)

        # Scroll, hogy több kártya betöltődjön
        await page.mouse.wheel(0, 5000)
        await page.wait_for_timeout(3000)

        a_links = await page.query_selector_all("a[data-cy='advert-card-link']") # linkek itt vannak a html-ben
        print(f"Talált linkek száma: {len(a_links)}")

        for a in a_links:
            href = await a.get_attribute("href")
            if href:
                links.append(_BASE_URL_ + "/" + href)

        await browser.close()
        return links

In [78]:
url=url_creator(place='pest-megye',type='haz')
print(url)
zenga_links=[]

# Futtatás
for i in range(5):
    print(f"Page: {i}")
    zenga_page = await scrape_zenga_links(start_url=url,page=i)
    zenga_links.extend(zenga_page)
# Kiírás
for i, link in enumerate(zenga_links, 1):
    print(f"{i}. {link}")


https://www.zenga.hu/pest-megye+elado+haz?page=
Page: 0
Talált linkek száma: 20
Page: 1
Talált linkek száma: 20
Page: 2
Talált linkek száma: 20
Page: 3
Talált linkek száma: 20
Page: 4
Talált linkek száma: 20
1. https://www.zenga.hu//ingatlan/elado-csaladi-haz-gyal/8538660?page=1&pos=1&cr=1000
2. https://www.zenga.hu//ingatlan/elado-csaladi-haz-rackeve/8568444?page=1&pos=2&cr=600
3. https://www.zenga.hu//ingatlan/elado-csaladi-haz-erd-parkvaros/8382804?page=1&pos=3&cr=500
4. https://www.zenga.hu//ingatlan/elado-csaladi-haz-szodliget/8172381?page=1&pos=4&cr=500
5. https://www.zenga.hu//ingatlan/elado-csaladi-haz-godollo/8440880?page=1&pos=5&cr=500
6. https://www.zenga.hu//ingatlan/elado-ikerhaz-urom/8618715?page=1&pos=6&cr=500
7. https://www.zenga.hu//ingatlan/elado-csaladi-haz-erd/8578168?page=1&pos=7&cr=500
8. https://www.zenga.hu//ingatlan/elado-csaladi-haz-erd/8033703?page=1&pos=8&cr=450
9. https://www.zenga.hu//ingatlan/elado-csaladi-haz-szentendre/8291448?page=1&pos=9&cr=450
10. ht

In [79]:
df=pd.DataFrame(zenga_links,columns=['link'])

In [80]:
csv_path='/content/drive/MyDrive/Zenga/'

In [81]:
df.to_csv(csv_path+'links_250727.csv',index=False)

In [87]:
async def scrape_ad_text(url):
    results = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()


        try:
            print(f"🔗 {url}")
            await page.goto(url, timeout=20000)
            try:
                await page.get_by_text("Összes süti engedélyezése").click()
                print("Süti elfogadva")
            except:
                print("Süti gomb nem jelent meg")


            await page.wait_for_timeout(7000)
            #await page.screenshot(path="screenshot.png")  # DEBUG
            desc_element = await page.query_selector("span[data-cy='advert-details-description']")
            desc_text = await desc_element.inner_text() if desc_element else "Nincs szöveg"
            print(f"Szöveg: {desc_text} ")

            price_element = await page.query_selector("div[data-cy='advert-details-price']")
            price_text = await price_element.inner_text() if desc_element else "Nincs ár"
            print(f"Ár: {price_text} ")


            results.append({"url": url, "desc": desc_text, "price":price_text})
        except Exception as e:
            print(f"Hiba: {str(e)}")
            results.append({"url": url, "text": f"Hiba: {str(e)}"})

        await browser.close()

    return results  #return a list


In [88]:
def scrape_zenga_list(z_list_list):
    out_list=[]
    for url in z_list_list:
        url_data = asyncio.run(scrape_ad_text(url))
        print(url_data)
        out_list.extend(url_data)
    return out_list

In [89]:
zenga_data=scrape_zenga_list(zenga_links)

🔗 https://www.zenga.hu//ingatlan/elado-csaladi-haz-gyal/8538660?page=1&pos=1&cr=1000
Süti elfogadva
Szöveg: Tágas, napfényes családi otthon Gyál csendes, zöldövezeti részén!Eladó egy 120 nm-es, jól karbantartott családi ház Gyálon, kiváló közlekedéssel és infrastruktúrával. Az ingatlan ideális többgyermekes családoknak is: 3 félszoba és 2 teljes méretű szoba, valamint nappali, étkező, konyha, és 2 fürdőszoba biztosítja a kényelmet – az egyik káddal, a másik zuhanyzóval felszerelt. Emellett egy tágas kamra is rendelkezésre áll.Az 583 nm-es, parkosítható telken fedett terasz (16 nm), gyűrűs kút, fedett és nyitott tárolók, valamint fedett kukatároló is található. Az udvaron 2 autó számára van parkolási lehetőség.Fűtés: gázcirkó és vegyes tüzelésű kazán, a nappaliban hűtő-fűtő klíma gondoskodik a kellemes hőmérsékletről nyáron is.A ház 2018-ban felújításon esett át: a vezetékek teljes cseréje, műanyag nyílászárók, új beltéri ajtók kerültek beépítésre. Egy festés után azonnal költözhető!A k

In [90]:
df=pd.DataFrame(zenga_data)
csv_path='/content/drive/MyDrive/Zenga/'
df.to_csv(csv_path+'zenga_data.csv',index=False)

In [91]:
from google.colab import userdata
key=userdata.get('OpenAi_API')

In [92]:
szoveg=df.desc[0]

In [77]:
1+"error_point" # A következő rész a AI prompt beállításait tartalmazza

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [93]:
szoveg='''
Budafok szép panorámás részén a Rózsavölgy feletti dombon, csendes mellékutcában eladó egy bájos kis ház 520 négyzetméteres telken.

A kis ház 35 négyzetméteres, egy légtérben van kialakítva benne a nappali, melyet részben nyitott válaszfallal szeparáltak el a konyha-étkező résztől. A zuhanyzós fürdő és a WC egy helyiségben található.

A víz, az áram és az internet be van kötve a házba. Gáz és csatorna az utcában. A csatorna bevezetését már kezdeményezte a szomszédos telek tulajdonosa.

A telken áll egy másik kis házikó is, ami jelenleg egy 9 négyzetméteres szobaként funkcionál.

A kert szép és gondozott, sok gyümölcsfával. Nagyobb mérete miatt sok lehetőséget rejt. Kialakítható rajta több nagy magaságyás, építhető kerti medence vagy finn szauna is.

További információkért és időpont egyeztetésért kérem hívjon telefononn.
Hitellel vásárlás esetén bankfüggetlen és DÍJMENTES hiteltanácsadó közreműködését biztosítjuk, aki a teljes folyamaton végigkíséri Önt.
Az adásvétel biztonságos és gördülékeny bonyolítása érdekében tapasztalt ingatlan szakjogászaink szolgáltatását is ajánlom.
'''

In [95]:
def create_prompt(szoveg):
    prompt_message=[
            {"role": "system",
             "content": f'''
                    Te egy ingatlanhirdetés-elemző nyelvész vagy, komoly ingatlanhirdetési tapasztalattal.
                    Célod, hogy az ingatlan hirdetések tökéletesek legyenek ezért kidolgoztál egy hirdetés értékelési szempontrendszert.
                    Ezek a szempontok alapján tökéletes értékelést tudsz adni az adott hirdetés szövege alapján.
                    A szempontok:
                        1. Érthetőség
                        2. Részletesség / információtartalom
                        3. Szerkezet, logikai felépítés
                        4. Célcsoport megszólítása
                        5. Stílus és nyelvhelyesség
                        6. Előnyök kiemelése
                        7. Negatívumok őszinte kezelése
                        8. Eladásra ösztönzés'''},
            {"role": "user",
             "content": """
                            Értékeld az alábbi ingatlanhirdetés szöveget és az értékelést a scoring mezőbe helyezzed el.
                            Az alábbi szempontok alapján 1–5-ig pontozzad, és hozz létre egy összesített eredményt  1-8 közötti szempontok értékelésének átlagaként ez legyen a 9. Összesítés :
                            Majd a tudásod alapján adj egy alternatív javaslatot az eredeti szöveg javítására, hogy a lehető legjobban megfeleljen a szempontoknak.
                            Az alternatív javaslatot strukturáld, és tördeld a jobb érthetőség érdekében, de markdown formátumot ne használj!
                            Fontos, hogy a saját szempontrendszered szerint az alternatív javaslatod értékelése jobb legyen az eredeti értékelésnél!
                            A válaszod csak érvényes JSON formátumban legyen, pontosan az alábbi struktúrában:

                            {"scoring":
                                {

                                    "Érthetőség": <szám>,
                                    "Részletesség": <szám>,
                                    "Szerkezet": <szám>,
                                    "Célcsoport": <szám>,
                                    "Stílus": <szám>,
                                    "Előnyök": <szám>,
                                    "Negatívumok": <szám>,
                                    "Ösztönzés": <szám>,
                                    "Összesítés": <szám.tizedes>"
                                },
                            "proposal":"Ide kerüljön a javított szöveg javaslatod"



                    A HIRDETÉS SZÖVEGE:
                        """+f" {szoveg}"}
        ]
    return prompt_message


In [96]:
prompt_message=create_prompt(szoveg)


In [97]:
def get_response(szoveg:str):
    client = OpenAI(api_key=key)
    prompt_message=create_prompt(szoveg)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=prompt_message,
        temperature=0.5,
        max_tokens=2900
    )

    content = response.choices[0].message.content

    # a gpt-4 így adja vissza a json választ, kiszedjük a jsont a felesleges keretből
    if content.startswith("```json"):
        content = content.strip("`").lstrip("json").strip()

    #
    out_dict = json.loads(content)

    return(out_dict)

In [98]:
resp_data=get_response(prompt_message)

In [99]:
print(resp_data)

{'scoring': {'Érthetőség': 4, 'Részletesség': 4, 'Szerkezet': 3, 'Célcsoport': 3, 'Stílus': 4, 'Előnyök': 4, 'Negatívumok': 3, 'Ösztönzés': 4, 'Összesítés': 3.6}, 'proposal': 'Eladásra kínálunk egy bájos kis házat Budafok szép panorámás részén, a Rózsavölgy feletti dombon, egy csendes mellékutcában. Az ingatlan 520 négyzetméteres telken helyezkedik el.\n\nA ház alapterülete 35 négyzetméter, ahol egy légtérben található a nappali, amelyet részben nyitott válaszfallal választottak el a konyha-étkező résztől. A fürdőszoba zuhanyzóval és WC-vel egy helyiségben található.\n\nA házba víz, áram és internet is be van kötve. A gáz és csatorna az utcában elérhető, a csatorna bevezetését már kezdeményezte a szomszédos telek tulajdonosa.\n\nA telken található még egy 9 négyzetméteres kis házikó, amely jelenleg szobaként funkcionál.\n\nA kert gondozott és tele van gyümölcsfákkal, így számos lehetőséget kínál: kialakítható rajta több nagy magaságyás, építhető kerti medence vagy finn szauna is.\n\nTo

In [100]:
resp_data["scoring"]

{'Érthetőség': 4,
 'Részletesség': 4,
 'Szerkezet': 3,
 'Célcsoport': 3,
 'Stílus': 4,
 'Előnyök': 4,
 'Negatívumok': 3,
 'Ösztönzés': 4,
 'Összesítés': 3.6}

In [101]:
resp_data["proposal"]

'Eladásra kínálunk egy bájos kis házat Budafok szép panorámás részén, a Rózsavölgy feletti dombon, egy csendes mellékutcában. Az ingatlan 520 négyzetméteres telken helyezkedik el.\n\nA ház alapterülete 35 négyzetméter, ahol egy légtérben található a nappali, amelyet részben nyitott válaszfallal választottak el a konyha-étkező résztől. A fürdőszoba zuhanyzóval és WC-vel egy helyiségben található.\n\nA házba víz, áram és internet is be van kötve. A gáz és csatorna az utcában elérhető, a csatorna bevezetését már kezdeményezte a szomszédos telek tulajdonosa.\n\nA telken található még egy 9 négyzetméteres kis házikó, amely jelenleg szobaként funkcionál.\n\nA kert gondozott és tele van gyümölcsfákkal, így számos lehetőséget kínál: kialakítható rajta több nagy magaságyás, építhető kerti medence vagy finn szauna is.\n\nTovábbi információkért és időpont egyeztetésért kérem, hívjon telefonon. Hitellel történő vásárlás esetén bankfüggetlen és díjmentes hiteltanácsadó közreműködését biztosítjuk, a

Eladásra kínálunk egy bájos kis házat Budafok szép panorámás részén, a Rózsavölgy feletti dombon, egy csendes mellékutcában. Az ingatlan 520 négyzetméteres telken helyezkedik el.

A ház alapterülete 35 négyzetméter, ahol egy légtérben található a nappali, amelyet részben nyitott válaszfallal választottak el a konyha-étkező résztől. A fürdőszoba zuhanyzóval és WC-vel egy helyiségben található.

A házba víz, áram és internet is be van kötve. A gáz és csatorna az utcában elérhető, a csatorna bevezetését már kezdeményezte a szomszédos telek tulajdonosa.

A telken található még egy 9 négyzetméteres kis házikó, amely jelenleg szobaként funkcionál.

A kert gondozott és tele van gyümölcsfákkal, így számos lehetőséget kínál: kialakítható rajta több nagy magaságyás, építhető kerti medence vagy finn szauna is.

További információkért és időpont egyeztetésért kérem, hívjon telefonon. Hitellel történő vásárlás esetén bankfüggetlen és díjmentes hiteltanácsadó közreműködését biztosítjuk, aki a teljes folyamaton végigkíséri Önt. Az adásvétel biztonságos és gördülékeny lebonyolítása érdekében tapasztalt ingatlan szakjogászaink szolgáltatását is ajánljuk.